In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
cwd = os.getcwd()
print(cwd)

C:\Users\bhoom\Earth Science NCSSM\EarthScienceSVSM GitHub


In [2]:
#2017 Data
df = pd.read_csv('EDA Analysis Folder/SRER_2017_training_bi.csv', index_col=0, parse_dates=True)
df.head()
df.isnull().sum()
df.dropna(axis=0,inplace=True)

In [3]:
import statsmodels.formula.api as smf

In [4]:
formula = 'Veg_class ~ CH_mean+ARVI_max+ARVI_mean+ARVI_med+EVI_max+EVI_mean+EVI_med+NDVI_max+NDVI_mean+NDVI_med+SAVI_max+SAVI_mean+SAVI_med'

In [5]:
#running a logistic regression rather than another type of linear
model_log = smf.glm(formula = formula, data=df, family=sm.families.Binomial())
result = model_log.fit()
print(result.summary())

                              Generalized Linear Model Regression Results                               
Dep. Variable:     ['Veg_class[non-woody]', 'Veg_class[woody]']   No. Observations:                 4339
Model:                                                      GLM   Df Residuals:                     4325
Model Family:                                          Binomial   Df Model:                           13
Link Function:                                            logit   Scale:                          1.0000
Method:                                                    IRLS   Log-Likelihood:                -2541.8
Date:                                          Wed, 06 Jul 2022   Deviance:                       5083.6
Time:                                                  15:39:00   Pearson chi2:                 2.66e+05
No. Iterations:                                               8                                         
Covariance Type:                                      n

In [6]:
print("Coefficeients")
print(result.params)
print()
print("p-Values")
print(result.pvalues)
print()
print("Dependent variables")
print(result.model.endog_names)

Coefficeients
Intercept      9.112032
CH_mean       -8.196147
ARVI_max      -2.169899
ARVI_mean     36.606140
ARVI_med       5.901491
EVI_max       39.854487
EVI_mean    -188.159328
EVI_med       48.455148
NDVI_max      -1.750538
NDVI_mean    -54.020462
NDVI_med     -10.892938
SAVI_max     -34.329641
SAVI_mean    217.822560
SAVI_med     -54.479211
dtype: float64

p-Values
Intercept    2.860077e-07
CH_mean      6.487483e-13
ARVI_max     7.296996e-01
ARVI_mean    2.481022e-02
ARVI_med     5.001155e-01
EVI_max      9.797461e-03
EVI_mean     5.129502e-05
EVI_med      8.266144e-02
NDVI_max     8.392977e-01
NDVI_mean    1.517295e-02
NDVI_med     3.403718e-01
SAVI_max     6.710801e-02
SAVI_mean    5.723029e-05
SAVI_med     8.134798e-02
dtype: float64

Dependent variables
['Veg_class[non-woody]', 'Veg_class[woody]']


In [7]:
predictions = result.predict()
print(predictions[0:10])

[0.13038227 0.28537094 0.66972367 0.61605885 0.53084829 0.45466805
 0.37682633 0.77316985 0.2905401  0.29999304]


In [8]:
print(np.column_stack((df.loc[:,"Veg_class"], result.model.endog)))

[['woody' 0.0]
 ['non-woody' 1.0]
 ['non-woody' 1.0]
 ...
 ['woody' 0.0]
 ['woody' 0.0]
 ['woody' 0.0]]


In [9]:
predictions_nominal = [ "woody" if x < 0.5 else "non-woody" for x in predictions]
print(predictions_nominal[1:10])

['woody', 'non-woody', 'non-woody', 'non-woody', 'woody', 'woody', 'non-woody', 'woody', 'woody']


In [10]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(df["Veg_class"], 
                       predictions_nominal))

[[1497  591]
 [ 782 1469]]


In [11]:
df["Veg_class"].value_counts()

woody        2251
non-woody    2088
Name: Veg_class, dtype: int64

In [12]:
print(classification_report(df["Veg_class"], predictions_nominal, digits = 3))
#look at 14

              precision    recall  f1-score   support

   non-woody      0.657     0.717     0.686      2088
       woody      0.713     0.653     0.682      2251

    accuracy                          0.684      4339
   macro avg      0.685     0.685     0.684      4339
weighted avg      0.686     0.684     0.683      4339



In [17]:
data = pd.read_csv('EDA Analysis Folder/SRER21_dataset_v1.csv', index_col=0, parse_dates=True)
data.isnull().sum()
data.dropna(axis=0,inplace=True)
x_train = df[:]
y_train = df['Veg_class']

x_test = data[:]
y_test = data['Veg_class']

In [18]:
model = smf.glm(formula = formula, data = x_train, family = sm.families.Binomial())
result = model.fit()
print(result.summary())

                              Generalized Linear Model Regression Results                               
Dep. Variable:     ['Veg_class[non-woody]', 'Veg_class[woody]']   No. Observations:                 4339
Model:                                                      GLM   Df Residuals:                     4325
Model Family:                                          Binomial   Df Model:                           13
Link Function:                                            logit   Scale:                          1.0000
Method:                                                    IRLS   Log-Likelihood:                -2541.8
Date:                                          Wed, 06 Jul 2022   Deviance:                       5083.6
Time:                                                  15:42:35   Pearson chi2:                 2.66e+05
No. Iterations:                                               8                                         
Covariance Type:                                      n

In [20]:
predictions = result.predict(x_test)
print(predictions)
predictions_nominal = [ "woody" if x < 0.5 else "non-woody" for x in predictions]
print(classification_report(y_test, predictions_nominal, digits = 3))

OID_
2       0.000182
3       0.000022
4       0.000116
5       0.000089
6       0.535308
          ...   
7761    0.000749
7762    0.000054
7763    0.005361
7766    0.001169
7767    0.001433
Length: 6805, dtype: float64
              precision    recall  f1-score   support

   non-woody      0.850     0.323     0.468      2230
       woody      0.747     0.972     0.845      4575

    accuracy                          0.759      6805
   macro avg      0.798     0.648     0.656      6805
weighted avg      0.780     0.759     0.721      6805



In [ ]:
#look at 17